In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime

In [ ]:
con = sqlite3.connect(':memory:')

In [ ]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [ ]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [ ]:
pd.read_sql(
    '''
    SELECT CustomerName,Country,Address FROM Customers cust
    where
    cust.Country in ('Germany','France')
    or
    cust.City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [ ]:
pd.read_sql(
    '''
    SELECT  Country,count(CustomerName) FROM Customers cust
    group by Country
    order by count(CustomerName) desc
    LIMIT 3

    ''',
    con,
)

,Country,count(CustomerName)
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [32]:
pd.read_sql(
    '''

    SELECT shi.ShipperName, o.OrderDate
    FROM Shippers shi
    join Orders o on o.ShipperID = shi.ShipperID
    order by o.OrderDate desc
    limit 10

    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,31.12.1996
1,Speedy Express,31.10.1996
2,Speedy Express,31.07.1996
3,United Package,31.01.1997
4,Federal Shipping,30.12.1996
5,United Package,30.10.1996
6,Speedy Express,30.09.1996
7,United Package,30.08.1996
8,Federal Shipping,30.07.1996
9,Speedy Express,30.01.1997


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [33]:
pd.read_sql(
    '''
    SELECT p.ProductName,p.Price,o.OrderID,od.Quantity,
      max(p.Price*od.Quantity) as Commission
     FROM Orders o
    join OrderDetails od on od.OrderID = o.OrderID
    join Products p on p.ProductID = od.ProductID
    order by Commission desc
    ''',
    con,
)

,ProductName,Price,OrderID,Quantity,Commission
0,Côte de Blaye,263.5,10353,50,13175.0


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [34]:
pd.read_sql(
    '''
    SELECT  pro.productName,ods.Quantity,ods.OrderID
    FROM Products pro
    join OrderDetails ods on ods.productID=pro.productId
    where  pro.productName in
    (
    SELECT p.productName--,sum(od.Quantity)
    FROM Products p
    left join OrderDetails od on od.productID=p.productId
    group by p.productName
    order by sum(od.Quantity) desc
    LIMIT 1
    )
    ''',
    con,
)

,ProductName,Quantity,OrderID
0,Gorgonzola Telino,20,10253
1,Gorgonzola Telino,40,10272
2,Gorgonzola Telino,15,10273
3,Gorgonzola Telino,4,10325
4,Gorgonzola Telino,25,10335
5,Gorgonzola Telino,56,10342
6,Gorgonzola Telino,30,10356
7,Gorgonzola Telino,70,10359
8,Gorgonzola Telino,20,10363
9,Gorgonzola Telino,30,10374


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [35]:
pd.read_sql(

'''

SELECT sup.SupplierName, count(o.OrderID),sup.Country,sup.ContactName,sup.Phone FROM Suppliers sup
left join Products p on p.SupplierID = sup.SupplierID
left join OrderDetails od on od.ProductID=p.ProductID
left join Orders o on o.OrderID=od.OrderID
group by sup.SupplierID
Order by count(o.OrderID) desc
LIMIT 5

''',

con,

)

,SupplierName,count(o.OrderID),Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,42,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",40,Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",33,UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,32,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,31,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [36]:
pd.read_sql(
    '''
    SELECT cat.CategoryName,sum(p.price),
    cus.Country FROM Categories cat
    left join Products p on p.CategoryID=cat.CategoryId
    left join OrderDetails od on od.ProductID=p.productID
    left join Orders o on o.OrderID=od.OrderID
    left join Customers cus on cus.CustomerID=o.CustomerID
    where cus.Country = 'Brazil'
    group by cat.CategoryName,cus.Country
    order by sum(p.price) desc
    LIMIT 1
    ''',
    con,
)

,CategoryName,sum(p.price),Country
0,Beverages,405.75,Brazil


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [37]:
pd.read_sql(
    '''
    SELECT
    o.orderID,
    max(od.Quantity*p.price) as maxOrderPrice,
    min(od.Quantity*p.price) as minOrderPrice,
    (max(od.Quantity*p.price) - min(od.Quantity*p.price)) as difference
    FROM Customers cus
    left join Orders o on o.CustomerID=cus.CustomerID
    left join OrderDetails od on od.OrderID=o.OrderID
    left join Products p on p.ProductID=od.ProductID
    where cus.Country = 'USA'
    ''',
    con,
)

,OrderID,maxOrderPrice,minOrderPrice,difference
0,10307,5270.0,37.5,5232.5


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [38]:
pd.read_sql(
    '''
   SELECT e.employeeID,
    (e.FirstName || ' ' || e.LastName) as FIO,
    count(OrderID)
    --TIMESTAMPDIFF(year, e.BirthDate, CURRENT_TIMESTAMP) AS year--,
       FROM Employees e
    left join Orders o on o.EmployeeID=e.EmployeeID
    group by e.employeeID
    order by e.BirthDate desc
    LIMIT 3
     ''',
    con,
)



,EmployeeID,FIO,"count(OrderID)\n --TIMESTAMPDIFF(year, e.BirthDate, CURRENT_TIMESTAMP) AS year--,"
0,3,Janet Leverling,31
1,7,Robert King,14
2,4,Margaret Peacock,40


10. Сколько банок крабового мяса всего было заказано.

In [39]:
pd.read_sql(
    '''
    SELECT p.ProductName,sum(od.Quantity) FROM Products p
    join OrderDetails od on od.productID=p.productId
    join Orders o on o.OrderID=od.OrderID
    where p.ProductName = 'Boston Crab Meat'
    ''',
    con,
)

,ProductName,sum(od.Quantity)
0,Boston Crab Meat,256
